In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib

import TAPI

def iterar_entre_fechas(fecha_desde, fecha_hasta):
    fecha_actual = datetime.strptime(fecha_desde, "%Y-%m-%dT%H:%M:%S.%f%z")
    fecha_fin = datetime.strptime(fecha_hasta, "%Y-%m-%dT%H:%M:%S.%f%z")

    # Asegurarse de que fecha_actual sea exactamente a la medianoche
    fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)

    while fecha_actual <= fecha_fin:
        fecha_siguiente = fecha_actual + timedelta(hours=23, minutes=59)
        yield fecha_actual, fecha_siguiente
        # Añadir un día para la próxima iteración
        fecha_actual += timedelta(days=1)

In [39]:
#from TAPI.ultimodocumento import ultimo_dia_CAMM



In [2]:
def  ultimo_dia_CAMM():

    ultimafecha = requests.get("https://api.cammesa.com/pub-svc/public/obtieneFechaUltimoDocumento?nemo=PARTE_POST_OPERATIVO")

    #fecha = str(ultimafecha.text[10:-2])
    fecha = ultimafecha.text[10:-2]

    
    
    return fecha

a = ultimo_dia_CAMM()

In [3]:
a

'2024-03-24'

In [37]:
ultimafecha = requests.get("https://api.cammesa.com/pub-svc/public/obtieneFechaUltimoDocumento?nemo=PARTE_POST_OPERATIVO")

#fecha = str(ultimafecha.text[10:-2])
fecha = ultimafecha.text[10:-2]

In [38]:
fecha

'2024-03-24'